In [5]:
from modules.base import *

#### 1. override : state는 기본적으로 override 됨.

In [8]:
class TypedDictState(TypedDict):
    cnt: int
    
@trace_function(enable_print=True)
def node_1(state):
    return {"cnt": state['cnt'] + 1}

builder = StateGraph(TypedDictState)
builder.add_node("node_1", node_1)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)
graph = builder.compile()
graph.invoke({"cnt":1})


🚀 Passing Through [node_1] ..

#### [Input State]
  args: ({'cnt': 1},)
  kwargs: {}

#### [Output State]
  result: {'cnt': 2}


{'cnt': 2}

#### 2. append : append 되는 형태로 하면, 멀티턴 쉽게 구현 가능
- 단순 state 일 경우 `from operator import add` 사용
- messages 일 경우 `from langgraph.graph.message import add_messages` 사용

add 사용

In [12]:
class State(TypedDict): 
    cnt: Annotated[list[int], add]

@trace_function(enable_print=True)
def node_1(state):
    return {"cnt": [state['cnt'][0] + 1]}

builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)
graph = builder.compile()
graph.invoke({"cnt":[1]})


🚀 Passing Through [node_1] ..

#### [Input State]
  args: ({'cnt': [1]},)
  kwargs: {}

#### [Output State]
  result: {'cnt': [2]}


{'cnt': [1, 2]}

add_messages 사용
- 여기서 중요한점은, add_messages를 추가한 MessagesState 클래스를 직접 선언해도되지만
- 사전에 정의된 Component를 사용해도됨 -> `from langgraph.graph import MessagesState`

In [20]:
# option 1 : 직접 정의
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# option 2 : 사전에 정의된 Component
# from langgraph.graph import MessagesState

# option 3 : 상속받기 (아래처럼 상속받으면, messages 와 summary 사용 가능)
# class State(MessagesState):
#     summary: str

@trace_function(enable_print=True)
def node_1(state:MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}

@trace_function(enable_print=True)
def node_2(state:MessagesState):
    return {"messages": [llm.invoke(state["messages"] + [HumanMessage(content="what is my name?")])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", END)
graph = builder.compile()
result = graph.invoke({"messages":"Hi, this is changwoo."})


🚀 Passing Through [node_1] ..

#### [Input State]
  args: ({'messages': [HumanMessage(content='Hi, this is changwoo.', additional_kwargs={}, response_metadata={}, id='cc1d396f-da37-4946-8827-71a963feb551')]},)
  kwargs: {}

#### [Output State]
  result: {'messages': [AIMessage(content='Hello Changwoo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fca5acb-32d4-4024-b19f-5c35873f792c-0', usage_metadata={'input_tokens': 14, 'output_tokens': 11, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': 

#### add_messages 이해
- id 값이 없을 경우 무조건 append
- id 값이 있을 경우, 해당 id 값이 있는 경우 overwrite

In [21]:
# without id -> only append
initial_messages = [AIMessage(content="Hello! How can I assist you?", 
                              name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", 
                                 name="Lance")]
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", 
                        name="Model")
add_messages(initial_messages , new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='72e28843-6de0-43af-8d8b-f1e49956c13b'),
 HumanMessage(content="I'm looking for information on marine biology.", additional_kwargs={}, response_metadata={}, name='Lance', id='dae14217-f80a-45d6-affd-e05805882e17'),
 AIMessage(content='Sure, I can help with that. What specifically are you interested in?', additional_kwargs={}, response_metadata={}, name='Model', id='fcf96012-32bf-4b1d-b1de-e727328d9847')]

In [22]:
# with id -> append and overwrite
initial_messages = [AIMessage(content="Hello! How can I assist you?", 
                              name="Model", 
                              id="1"),
                    HumanMessage(content="I'm looking for information on marine biology.", 
                                 name="Lance", 
                                 id="2")]
new_message = HumanMessage(content="I'm looking for information on whales, specifically", 
                           name="changwoo", 
                           id="2")
add_messages(initial_messages , new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='1'),
 HumanMessage(content="I'm looking for information on whales, specifically", additional_kwargs={}, response_metadata={}, name='changwoo', id='2')]

#### add_messages + RemoveMessage 이해
- add_messages 사용시, RemoveMessage 값을 넘기면 지울수있음.

In [23]:
messages = [AIMessage("Hi.", 
                      name="Bot", 
                      id="1")]
messages.append(HumanMessage("Hi.", 
                             name="Lance", 
                             id="2"))
messages.append(AIMessage("So you said you were researching ocean mammals?", 
                          name="Bot", 
                          id="3"))
messages.append(HumanMessage("Yes, I know about whales. But what others should I learn about?", 
                             name="Lance", 
                             id="4"))

delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]
add_messages(messages , delete_messages)

[AIMessage(content='So you said you were researching ocean mammals?', additional_kwargs={}, response_metadata={}, name='Bot', id='3'),
 HumanMessage(content='Yes, I know about whales. But what others should I learn about?', additional_kwargs={}, response_metadata={}, name='Lance', id='4')]

- filter_messages 를 통과한뒤, 4개의 메시지중 처음 2개 삭제됐음.

In [24]:
@trace_function(enable_print=True)
def filter_messages(state: MessagesState):
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"messages": delete_messages}

@trace_function(enable_print=True)
def chat_model_node(state: MessagesState):    
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(MessagesState)
builder.add_node("filter", filter_messages)
builder.add_node("chat_model", chat_model_node)
builder.add_edge(START, "filter")
builder.add_edge("filter", "chat_model")
builder.add_edge("chat_model", END)
graph = builder.compile()

In [25]:
output = graph.invoke({'messages': messages})
print('\n\n') 
for m in output['messages']:
    m.pretty_print()


🚀 Passing Through [filter_messages] ..

#### [Input State]
  args: ({'messages': [AIMessage(content='Hi.', additional_kwargs={}, response_metadata={}, name='Bot', id='1'), HumanMessage(content='Hi.', additional_kwargs={}, response_metadata={}, name='Lance', id='2'), AIMessage(content='So you said you were researching ocean mammals?', additional_kwargs={}, response_metadata={}, name='Bot', id='3'), HumanMessage(content='Yes, I know about whales. But what others should I learn about?', additional_kwargs={}, response_metadata={}, name='Lance', id='4')]},)
  kwargs: {}

#### [Output State]
  result: {'messages': [RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='1'), RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='2')]}

🚀 Passing Through [chat_model_node] ..

#### [Input State]
  args: ({'messages': [AIMessage(content='So you said you were researching ocean mammals?', additional_kwargs={}, response_metadata={}, name='Bot', id='3'), HumanM

#### 3. trim messages : truncate with max_tokens
- 전체 대화 텍스트에 대해 max_token 기준으로 자름.
- 아래 예시에서는 max_tokens=80으로 잘랐을경우, 최근 대화 2개만 남음.

In [26]:
test_messages = [HumanMessage(content="Hi, this is changwoo. How are you?"),
                 AIMessage(content="I'm fine, thank you. How can I help you?"),
                 HumanMessage(content="I'm looking for information on whales. What should I know about them?"),
                 AIMessage(content="Whales are fascinating creatures. They are the largest animals on Earth. They are found in all oceans, but they are most commonly found in the Pacific Ocean. They are known for their intelligence and their ability to communicate with each other.")]

messages = trim_messages(test_messages,
                         max_tokens=80,
                         strategy="last",
                         token_counter=ChatOpenAI(model="gpt-4o"),
                         allow_partial=False)

messages

[HumanMessage(content="I'm looking for information on whales. What should I know about them?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Whales are fascinating creatures. They are the largest animals on Earth. They are found in all oceans, but they are most commonly found in the Pacific Ocean. They are known for their intelligence and their ability to communicate with each other.', additional_kwargs={}, response_metadata={})]